# Your Automated CI/CD Pipeline

Congratulations on making it to the final stage of our foundational MLOps journey. You've mastered an incredible set of skills, piece by piece:

* You can manage your project's history with **Git**.
* You can collaborate and back up your work using **GitHub**.
* You've automated your code quality checks and tests with **GitHub Actions** and **Ruff**.
* You know how to create portable, reproducible environments with **Docker**.

Now, it's time to connect all these pieces. In this notebook, we'll transform your separate automated checks and manual Docker builds into a single, seamless pipeline. The goal is to create a professional workflow where every push to your main branch automatically checks your code, builds a new Docker image, and publishes it for the world to use. This is the core of modern CI/CD (Continuous Integration/Continuous Deployment).

---
## 1.&nbsp; The Goal: A Fully Automated Workflow 🎯

Here's what our final pipeline will do:

1.  **Trigger:** A developer (you!) pushes a new commit to the `main` branch.
2.  **Check:** GitHub Actions automatically runs your linter and your tests in parallel.
3.  **Gate:** If, and only if, both the linter and the tests pass, the workflow proceeds.
4.  **Build & Publish:** The workflow automatically builds a new Docker image from your `Dockerfile` and pushes it to your Docker Hub account.

This creates a powerful safety net. No code with failing tests or poor formatting will ever result in a published image. Let's build it.

---
## 2.&nbsp; Prerequisite: Storing Your Credentials Securely 🔐

Our workflow needs to log in to Docker Hub to publish an image, but we can never, ever write our passwords directly into our code or workflow files. This is a major security risk.

The professional solution is to use **GitHub Secrets**. These are encrypted environment variables that you can create for your repository. Only your GitHub Actions workflows can access them, keeping your credentials safe.

### 2.1. Create a Docker Hub Access Token

First, we'll create a special Access Token on Docker Hub. This is safer than using your main password because you can give it specific, limited permissions and revoke it at any time.

1.  Log in to your [Docker Hub](https://hub.docker.com/) account.
2.  Click on your username in the top-right corner and go to `Account Settings`.
3.  In the left sidebar, click on `Personal access tokens`.
4.  Click the `Generate new token` button.
5.  Give your token a descriptive name, like `github-actions-token`.
6.  Set its permissions to `Read & Write`. This is all our workflow needs.
7.  Click `Generate`.
8.  **Important:** Docker Hub will show you the token now. **Copy this token immediately and save it somewhere safe**, like a password manager. You will not be able to see it again after you close this window.


### 2.2. Add Your Secrets to GitHub

Now, let's store your Docker Hub username and your new token in your GitHub repository's secrets.

1.  Go to your `sentiment-analysis-project` repository on GitHub.
2.  Click on the `Settings` tab.
3.  In the left sidebar, expand `Secrets and variables` and click on `Actions`.
4.  Click the `New repository secret` button.
5.  Create the first secret:
    * Name: `DOCKERHUB_USERNAME`
    * Secret: Enter your Docker Hub username.
    * Click `Add secret`.
6.  Create the second secret:
    * Click `New repository secret` again.
    * Name: `DOCKERHUB_TOKEN`
    * Secret: Paste the Access Token you just created on Docker Hub.
    * Click `Add secret`.

Your credentials are now securely stored and ready for our workflow to use.

---
## 3.&nbsp; Building the Full Pipeline 🏗️

It's time to edit our workflow file. Open your `.github/workflows/ci.yml` file in VS Code. We're going to add a new job that depends on our existing jobs.

### 3.1. Add the `build-and-push` Job

Below your existing `test` job, add the following new job. Make sure the indentation is correct; `build-and-push` should be at the same level as your other jobs (`run-predict`, `test`, etc.).

In [ ]:
  build-and-push:
    runs-on: ubuntu-latest
    needs: [test, lint-and-format]

    steps:
      - name: Check out repository
        uses: actions/checkout@v4

      - name: Log in to Docker Hub
        uses: docker/login-action@v3
        with:
          username: ${{ secrets.DOCKERHUB_USERNAME }}
          password: ${{ secrets.DOCKERHUB_TOKEN }}

      - name: Build and push Docker image
        uses: docker/build-push-action@v5
        with:
          context: .
          push: true
          tags: ${{ secrets.DOCKERHUB_USERNAME }}/sentiment-analysis-app:latest

Let's break down the new parts:

* `needs: [test, lint-and-format]`: This is the most important new line. It tells GitHub Actions that this job depends on the `test` and `lint-and-format` jobs. It will not even start until both of those jobs have completed successfully. This is our quality gate.
* `uses: docker/login-action@v3`: This is a pre-made Action that securely logs into Docker Hub for us.
* `with: ...`: We pass our GitHub Secrets to the login action. The `${{ secrets.VAR_NAME }}` syntax is how you access secrets within a workflow.
* `uses: docker/build-push-action@v5`: This powerful action handles both building and pushing our image in a single step.
* `context: .`: Tells the action to find the `Dockerfile` in the root of our project.
* `push: true`: Tells the action to push the image to the registry after building it.
* `tags: ...`: This is how we name our image. We use our `DOCKERHUB_USERNAME` secret to ensure the image is tagged correctly for our account (e.g., `your-username/sentiment-analysis-app:latest`).

---
## 4.&nbsp; The Complete, Final Workflow 📜

After adding the new job, your entire `ci.yml` file should look like this. Double-check your indentation carefully, as YAML is very sensitive to it.

In [ ]:
name: CI

on:
  push:
    branches: [ main ]
    paths:
      - "src/**"
      - "tests/**"
      - "data/**"
      - "requirements.txt"
      - ".github/workflows/**"
  pull_request:
    paths:
      - "src/**"
      - "tests/**"
      - "data/**"
      - "requirements.txt"
      - ".github/workflows/**"

jobs:
  lint-and-format:
    runs-on: ubuntu-latest
    steps:
      - name: Check out repository
        uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.11"

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt

      - name: Run linter and formatter checks
        run: |
          ruff check .
          ruff format . --check

  test:
    runs-on: ubuntu-latest
    steps:
      - name: Check out repository
        uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.11"

      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt

      - name: Run tests with pytest
        run: pytest -v

  build-and-push:
    runs-on: ubuntu-latest
    needs: [test, lint-and-format]

    steps:
      - name: Check out repository
        uses: actions/checkout@v4

      - name: Log in to Docker Hub
        uses: docker/login-action@v3
        with:
          username: ${{ secrets.DOCKERHUB_USERNAME }}
          password: ${{ secrets.DOCKERHUB_TOKEN }}

      - name: Build and push Docker image
        uses: docker/build-push-action@v5
        with:
          context: .
          push: true
          tags: ${{ secrets.DOCKERHUB_USERNAME }}/sentiment-analysis-app:latest

---
## 5.&nbsp; Triggering and Verifying Your Pipeline 🚀

Now for the final test. Let's commit our new workflow and see it run automatically.

1.  In your VS Code terminal, commit your updated `.github/workflows/ci.yml` file.

In [ ]:
git add .github/workflows/ci.yml

In [ ]:
git commit -m "ci: Add Docker build and publish job"

2.  Push the commit to your `main` branch.

In [ ]:
git push

3.  Watch it run: Go to the **Actions** tab on your GitHub repository. You'll see your new workflow run has started. This time, the visual graph will show your `test` and `lint-and-format` jobs running in parallel. Only when they both finish with a green checkmark will the `build-and-push` job begin. The connecting lines on the graph visually represent the `needs` dependency you created.

4.  Verify on Docker Hub: Once the entire workflow is green, navigate to your profile on Docker Hub. You should see a new public repository named `sentiment-analysis-app`. Click on it, and you'll see your image, tagged as `latest`, successfully published.

You've done it! You now have a professional, automated CI/CD pipeline that ensures your code is always tested, clean, and ready for use.